In [1]:
from main import OPENAI_API_KEY
config_list = [{
    'model': 'gpt-4',
    'api_key': OPENAI_API_KEY,
}]
llm_copywriter ={
    "config_list": config_list

}
llm_marketer = {
    "seed": 42,  # seed for caching and reproducibility
    "config_list": config_list,  # a list of OpenAI API configurations
    "temperature": 0,
    "function": [{
        "name":"answer_uniswap_question",
        "description": "Ответ на любой Uniswape вопрос",
        "parameters": {
            "type": "object",
            "properties": {
                "question": {
                    "type": "string",
                    "description": "Вопрос, который следует задать согласно uniswap",
                },
            },
            "required": ["question"],
        },
    }], # temperature for sampling
    "use_cache": True,  # whether to use cache
}

In [42]:

from Promt_prototype.uniswap import answer_uniswap_question
import autogen


copywriter = autogen.AssistantAgent(
    name="Copywriter",
    max_consecutive_auto_reply=3, llm_config=llm_copywriter,
    system_message=f"Copywriter. Ты создаёшь рекламу для какого либо продукта",
)
marketer = autogen.UserProxyAgent(name="Marketer",
                         max_consecutive_auto_reply=3,
                         llm_config=llm_marketer,
                         system_message="Marketer. Ты анализируешь созданную рекламу Копирайтером и даёшь рекомендацию согласно full_documents")
# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="Admin",
    human_input_mode="NEVER",
    llm_config=llm_copywriter,
    max_consecutive_auto_reply=10,
    code_execution_config={
        "work_dir": ".",
    },
    system_message=f"Admin. Просто следит за общением ассистентов.",function_map={"answer_uniswap_question": answer_uniswap_question},
)

In [43]:
groupchat = autogen.GroupChat(agents=[user_proxy,copywriter,marketer],messages=[],max_round=3)
manager =autogen.GroupChatManager(groupchat=groupchat,llm_config=llm_copywriter)

In [44]:
user_proxy.initiate_chat(
    manager,
    message="""
            1. Copywriter напиши рекламу про гарри поттера. 
            2. Marketer проанализируй рекламу Копирайтера и дай рекомендации копирайтеру согласно full_documents
            3. Copywriter измени свой рекламный текст учитывая рекомендации маркетолога.
            4. Marketer снова проанализируй результат Копирайтера и дай рекомендации копирайтеру согласно full_documents.
            5. Copywriter снова измени свой рекламный текст учитывая рекомендации маркетолога.
            6. Marketer снова проанализируй результат Копирайтера и дай рекомендации копирайтеру согласно full_documents.
            7. Copywriter снова измени свой рекламный текст учитывая рекомендации маркетолога.
            8. Marketer снова проанализируй результат Копирайтера и дай рекомендации копирайтеру согласно full_documents.
            """,
)

Admin (to chat_manager):


            1. Copywriter напиши рекламу про гарри поттера. 
            2. Marketer проанализируй рекламу Копирайтера и дай рекомендации копирайтеру согласно full_documents
            3. Copywriter измени свой рекламный текст учитывая рекомендации маркетолога.
            4. Marketer снова проанализируй результат Копирайтера и дай рекомендации копирайтеру согласно full_documents.
            5. Copywriter снова измени свой рекламный текст учитывая рекомендации маркетолога.
            6. Marketer снова проанализируй результат Копирайтера и дай рекомендации копирайтеру согласно full_documents.
            7. Copywriter снова измени свой рекламный текст учитывая рекомендации маркетолога.
            8. Marketer снова проанализируй результат Копирайтера и дай рекомендации копирайтеру согласно full_documents.
            

--------------------------------------------------------------------------------
Copywriter (to chat_manager):

1. Copywriter: "Погрузитесь 

In [49]:
messages = user_proxy.chat_messages[manager]
print(messages)

[{'content': '\n            1. Copywriter напиши рекламу про гарри поттера. \n            2. Marketer проанализируй рекламу Копирайтера и дай рекомендации копирайтеру согласно full_documents\n            3. Copywriter измени свой рекламный текст учитывая рекомендации маркетолога.\n            4. Marketer снова проанализируй результат Копирайтера и дай рекомендации копирайтеру согласно full_documents.\n            5. Copywriter снова измени свой рекламный текст учитывая рекомендации маркетолога.\n            6. Marketer снова проанализируй результат Копирайтера и дай рекомендации копирайтеру согласно full_documents.\n            7. Copywriter снова измени свой рекламный текст учитывая рекомендации маркетолога.\n            8. Marketer снова проанализируй результат Копирайтера и дай рекомендации копирайтеру согласно full_documents.\n            ', 'role': 'assistant'}, {'content': '1. Copywriter: "Погрузитесь в волшебный мир Гарри Поттера, где магия становится реальностью. Откройте для с

In [ ]:
import gradio as gra
from Promt_prototype.manager import run_flow
# define gradio interface and other parameters
app = gra.Interface(fn=run_flow, inputs="text", outputs="text")
app.launch(
    # share=True
)